In [ ]:
from qepc.utils.paths import get_project_root
import sys

PROJECT_ROOT = get_project_root()
if str(PROJECT_ROOT) not in sys.path:
    sys.path.append(str(PROJECT_ROOT))

print('PROJECT_ROOT:', PROJECT_ROOT)


In [ ]:
from qepc.brain.games_loader import fetch_league_games, build_games_table
from qepc.brain.boxscores_loader import fetch_boxscores_for_games
from qepc.brain.state_vectors import build_rolling_player_state_vectors

season = "2023-24"

# 1) Games table
team_games = fetch_league_games(season)
games_df = build_games_table(team_games)

print("games_df rows:", len(games_df))
display(games_df.head())

# 2) Take a manageable subset of GAME_IDs to start
game_ids = games_df["GAME_ID"].unique().tolist()[:100]  # first 100 games

print("Number of GAME_IDs we'll use:", len(game_ids))

trad_df, _ = fetch_boxscores_for_games(game_ids, sleep_seconds=0.7, verbose=True)

print("trad_df shape:", trad_df.shape)
display(trad_df.head())


In [ ]:
from qepc.brain.state_vectors import build_rolling_player_state_vectors

rolling_state_df = build_rolling_player_state_vectors(
    boxscores_df=trad_df,
    games_df=games_df,
    window=10,
)

print("rolling_state_df shape:", rolling_state_df.shape)
display(rolling_state_df.head())


In [ ]:
from qepc.brain.players_loader import fetch_league_player_season_stats

season = "2023-24"

# Get the same season-level player stats we used before
player_stats_df = fetch_league_player_season_stats(season, measure_type="Base")

# Keep just the label columns we care about
label_cols = ["PLAYER_ID", "PLAYER_NAME", "TEAM_ID", "TEAM_ABBREVIATION"]
label_cols = [c for c in label_cols if c in player_stats_df.columns]

player_labels = player_stats_df[label_cols].drop_duplicates(subset=["PLAYER_ID"])

# Join labels onto the rolling state table
rolling_pretty = rolling_state_df.merge(
    player_labels,
    on="PLAYER_ID",
    how="left",
)

# Reorder columns so labels show up first
front_cols = ["PLAYER_NAME", "TEAM_ABBREVIATION", "PLAYER_ID", "TEAM_ID", "GAME_ID", "GAME_DATE"]
front_cols = [c for c in front_cols if c in rolling_pretty.columns]

other_cols = [c for c in rolling_pretty.columns if c not in front_cols]

rolling_pretty = rolling_pretty[front_cols + other_cols]

display(rolling_pretty.head())
